<h1 id="tocheading">TABLE OF CONTENTS</h1>
<div id="toc"></div>

**Updates to the table of contents are periodic, but run the cell below to first start or force an update.**

In [ ]:
macro javascript_str(s) display("text/javascript", s); end

javascript"""
$.getScript('make_table_of_contents.js')
"""

# Alex-style runs

In [ ]:
using ForwardDiff
using DiffBase
using MAT
using PyPlot


In [ ]:
include("pro_anti.jl")

In [ ]:
FarmName = "CARLOS_TEST"


#Define core model parameters
model_params = Dict(
:dt     =>  0.002,      # Changed hyper-parameter
:tau    =>  0.02,       # Changed hyper-parameter
:vW     =>  -1.7, #FIT parameter
:hW     =>  -1.7, #FIT parameter
:sW     =>  0.2,  #FIT parameter
:dW     =>  0,    #FIT parameter
:nsteps =>  2, 
:noise  =>  [], 
:sigma  =>  0.08, #FIT parameter
:input  =>  0,    #### Alex thinks this is out-dated 
:g_leak =>  1,          # Changed hyper-parameter
:U_rest =>  0,          # Changed hyper-parameter
:theta  =>  0.05,       # Changed hyper-parameter
:beta   =>  0.5,        # Changed hyper-parameter 
:sw     =>  0.2,  #### Alex thinks this is out-dated
:hw     =>  -1.7, #### Alex thinks this is out-dated
:vw     =>  -1.7, #### Alex thinks this is out-dated
:constant_excitation      => 0.19, #FIT  parameter
:anti_rule_strength       => 0.05,      # Changed hyper-parameter
:pro_rule_strength        => 0.05,      # Changed hyper-parameter
:target_period_excitation => 1,    #FIT  parameter
:right_light_excitation   => 0.5,  #FIT parameter
:right_light_pro_extra    => 0,
:const_add                => 0, 
:init_add                 => 0, 
:rule_and_delay_period    => 0.4,
:target_period            => 0.1,
:post_target_period       => 0.5,
:const_pro_bias           => 0,    #FIT parameter
:nPro                     => 100,
:nAnti                    => 100,
:theta1                   => 0.01,
:theta2                   => 0.15,
:start_pro                => [-0.5, -0.5, -0.5, -0.5],
:start_anti               => [-0.5, -0.5, -0.5, -0.5],
:opto_strength  => .7,             #FIT parameter
:opto_periods   => [-1.1 -1; 0 20;0 50; 50 100; 100 20],  
# The opto "conditions" correspond to the rows of opto_periods.
# all conditions are in seconds relative to start of the trial
# any value before 0 gets changed to 0
# any value after the end of trial gets changed to end of trial
# Special values allow for variable durations
#  20 codes for "end of trial"
#  -1 codes for "start of trial"
# 100 codes for "end of rule and delay period"
# 200 codes for "end of target period"
# -------------------------------------------------------------
# first column is frachit Pro, next column is Anti, rows are conditions
# Actual Opto targets
#:opto_targets   => [.75 .73;.77 .58;.75 .74; .72 .66;.73 .75] 
# Fake Targets
:opto_targets => [.9 .7; .9 .5; .9 .7; .9 .5; .9 .7]  
);

# ======= ARGUMENTS AND SEED VALUES:
args = ["sW", "vW", "hW", "dW", "constant_excitation", "right_light_excitation", "target_period_excitation", "const_pro_bias", "sigma","opto_strength"];
seed = [0.001,  -1.58,   -0.05,  0.001,    0.001,                0.6,                       0.001,                        0.0427,              0.05, .9];   
# ======= BOUNDING BOX:
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0 4], :const_pro_bias=>[-2 2],
:sigma=>[0.1 0.4],:opto_strength=>[0 1]);

# ======== SEARCH ZONE:
sbox = Dict(:sW=>[0 .5], :vW=>[-.5 .5], :hW=>[-.5 .5], :dW=>[-.5 .5],
:constant_excitation=>[-.5 .5], :right_light_excitation=>[0.05 .5], :target_period_excitation=>[0.001 .5],:const_pro_bias=>[-.5 .5], :sigma=>[0.1 .2],:opto_strength=>[.7 .99]);

# define a few hyper parameters
cbetas = [4e-3];
rule_and_delay_periods = [0.2];
post_target_periods    = [0.05];
num_eval_runs           = 1000;
num_optimize_iter       = 2000;
num_optimize_restarts   = 100;

# define base filename
fbasename = "FarmFields/farm_"*string(FarmName);

for i=1:num_optimize_restarts   # Iterate this many optimizations
for cb in cbetas                # Iterate over beta values, if there are multiple
    # figure out initial seed for random number generator
    sr = convert(Int64, round(time()))
    srand(sr);

    dista=rand();
    println("dista")
    println(dista)
    myseed1 = copy(seed);

    # get initial parameter values by sampling from sbox
    myseed = ForwardDiffZeros(length(args),1);
    for j=1:length(args)
        sym = Symbol(args[j])
        if haskey(sbox, sym)
#           myseed[j] = sbox[sym][1] + diff(sbox[sym],2)[1]*rand();
            myseed[j] = dista*(sbox[sym][1] + diff(sbox[sym],2)[1]*rand())+(1-dista)*myseed1[j];
        if myseed[j] > bbox[sym][2]
            myseed[j] =bbox[sym][2] - 0.1;
        elseif myseed[j] < bbox[sym][1]
            myseed[j] =bbox[sym][1]+0.1;
        end
        else
            myseed[j] = seed[j];
        end
    end

    # define opto function with just value output
    func =  (;params...) -> JJ(model_params[:nPro], model_params[:nAnti]; 
        rule_and_delay_periods=rule_and_delay_periods, 
        theta1=model_params[:theta1], theta2=model_params[:theta2], 
        post_target_periods=post_target_periods, seedrand=sr, cbeta=cb, 
        verbose=false, merge(model_params, Dict(params))...)[1]
    
    # run optimization with all parameters
    @printf("Going with seed = "); print_vector_g(myseed); print("\n")
    pars, traj, cost, cpm_traj = bbox_Hessian_keyword_minimization(myseed, args, bbox, func, start_eta = 1, tol=1e-12, verbose=true, verbose_every=10, maxiter=num_optimize_iter)
    @printf("Came out with cost %g and pars = ", cost); print_vector_g(pars); print("\n\n")

    # get gradient and hessian at end of optimization 
    value, grad, hess = keyword_vgh(func, args, pars)

    # define function with all outputs, evaluate on training noise
    t_standard_func =  (;params...) -> JJ_opto(model_params[:nPro], model_params[:nAnti]; rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], post_target_periods=post_target_periods, seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    # run opto model with all outputs, evaluate on training noise
    t_opto_scost, t_opto_scost1, t_opto_scost2, t_opto_hitsP,t_opto_hitsA, t_opto_diffsP, t_opto_diffsA, t_opto_bP, t_opto_bA = t_standard_func(;make_dict(args, pars, model_params)...)
    
   ## evaluate for long form info
    # reset random number generator for testing purposes
    test_sr = convert(Int64, round(time()))
    srand(test_sr); 

    # define function with all outputs, evaluate on test noise
    standard_func =  (;params...) -> JJ_opto(num_eval_runs, num_eval_runs; rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], post_target_periods=post_target_periods, seedrand=test_sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    # run opto model with all outputs, evaluate on test noise
    opto_scost, opto_scost1, opto_scost2, opto_hitsP,opto_hitsA, opto_diffsP, opto_diffsA, opto_bP, opto_bA = standard_func(;make_dict(args, pars, model_params)...)

   ## Save this run out to a file
    # get filename
    myfilename = next_file(fbasename, 4)
    myfilename = myfilename*".mat"
    # write file
    matwrite(myfilename, Dict("args"=>args, "myseed"=>myseed, "pars"=>pars, "traj"=>traj, "cost"=>cost, "cpm_traj"=>cpm_traj, "nPro"=>model_params[:nPro], "nAnti"=>model_params[:nAnti], "sr"=>sr, "cb"=>cb, "theta1"=>model_params[:theta1], "theta2"=>model_params[:theta2],"value"=>value,"grad"=>grad, "hess"=>hess, "model_params"=>ascii_key_ize(model_params), "bbox"=>ascii_key_ize(bbox), "sbox"=>ascii_key_ize(sbox), "rule_and_delay_periods"=>rule_and_delay_periods, "post_target_periods"=>post_target_periods, "opto_scost"=>opto_scost, "opto_scost1"=>opto_scost1, "opto_scost2"=>opto_scost2, "opto_hitsP"=>opto_hitsP, "opto_hitsA"=>opto_hitsA, "opto_diffsP"=>opto_diffsP, "opto_diffsA"=>opto_diffsA,"test_sr"=>test_sr,"opto_bP"=>opto_bP, "opto_bA"=>opto_bA, "t_opto_scost"=>t_opto_scost, "t_opto_scost1"=>t_opto_scost1, "t_opto_scost2"=>t_opto_scost2, "t_opto_hitsP"=>t_opto_hitsP, "t_opto_hitsA"=>t_opto_hitsA, "t_opto_diffsP"=>t_opto_diffsP, "t_opto_diffsA"=>t_opto_diffsA,"t_opto_bP"=>t_opto_bP, "t_opto_bA"=>t_opto_bA,"dista"=>dista  ))
end
end



In [ ]:
using HypothesisTests

In [ ]:
N = 20
x = 1+0.2*randn(N,1); y = x - 0.05 + 0.1*randn(N,1);


using PyPlot
pygui(true)

figure(1); clf();
subplot(1,2,1)

for i=1:N
    plot([1, 2], [x[i], y[i]], "b.-")
end

p = pvalue(OneSampleTTest(x[:], y[:]))

title("Paired t-test :  p<" * @sprintf("%.4g", p))


subplot(1,2,2)
plot(x-y, "b.")

# Marino's model in Marino's parameter settings

In [ ]:
#@include_me  scaling_up_dt.jl

# In its own cell so we can run it just once

include("pro_anti.jl")   # Loads all the necessary pre-requisites


In [ ]:
#@include_me  scaling_up_dt.jl

model_params, F, nPro, nAnti = load_run("farm_LD0003"; farmdir="goodfarms")

# mypars = merge(model_params, Dict(:post_target_periods=>0.05))

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA, proValls, antiValls, opto_fraction, pro_input, anti_input = 
JJ(model_params[:nPro], model_params[:nAnti]; verbose=true, model_details=true, model_params...);


epochs = ["control";"full";"rule";"delay";"choice"]
titles  = ["epochs" "hitP" "hitA" "diffP" "diffA" "hBP" "hBA"]

@printf("\n\n--- The original run:\n\n")

sleep(0.5)  # A pause just to make sure everything has printed out 
display([titles; epochs hP hA dP dA hBP hBA])


In [ ]:
#@include_me  scaling_up_dt.jl

# Now scale things to make dt and tau are much larger, and involve 0.25 x the timesteps

mypars = merge(model_params, Dict(:opto_times=>[0 0]))  # default is no opto

mypars = merge(model_params, Dict(
:dt=>0.024,
:tau=>0.09,
:rule_and_delay_period=>1.2,
:target_period=>0.3,
:post_target_period=>0.3,
:anti_rule_strength=>0.054,
# :post_target_period=>0.05, 
# :rule_and_delay_period=>0.8, 
# :opto_times=>[0 0],
# :opto_times=>["target_start/2" "target_start"],
# :opto_times=>["target_start", "target_end"],
:opto_strength=>0.85,
))# , :rule_and_delay_period=>0.8))


mypars[:rule_and_delay_periods] = [mypars[:rule_and_delay_period]]
mypars[:target_periods]         = [mypars[:target_period]]
mypars[:post_target_periods]    = [mypars[:post_target_period]]

mypars[:opto_periods] = [
    "trial_start"     "trial_start"  ;
    "trial_start"     "trial_end"    ; 
    "trial_start"     "target_start/2"  ;
    "target_start/2"  "target_start"  ;
    "target_start"    "target_end"
];


In [ ]:
#@include_me  scaling_up_dt.jl

# And run with these params.  

# We run 1000 trials of Pro and ANti each, just to get a good estimate of probability correct in each condition

mypars[:seedrand] = Int64(round(time()*1000))

ntrials = 1000

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA, proValls, antiValls, opto_fraction, pro_input, anti_input = 
JJ(ntrials, ntrials; verbose=true, model_details=true, mypars...);

@printf("\n\n--- The hand-modified run at dt=0.024 at 4x fewer timesteps than the original:\n\n")


sleep(0.5)  # A pause just to make sure everything has printed out 
display([titles; epochs hP hA dP dA hBP hBA])


In [ ]:
#@include_me  scaling_up_dt.jl

# Finally, display some trajectories:

pygui(true)

# This example is with opto during the delay

mypars = merge(mypars, Dict(
# :opto_times => ["target_start/2"    "target_start"],
:opto_times => [0    0],
:sigma=>0.01
))


proVs, antiVs, pro_fullV, anti_fullV, opto_fraction, pro_input, anti_input = run_ntrials(1000, 1000; 
    plot_list=[1:40;], profig=1, antifig=2, opto_units = 1:4, mypars...);

Np = size(proVs,2); Na = size(antiVs,2)
@printf("hBP = %g, hBA = %g\n", length(find(proVs[1,:] .>= proVs[4,:]))/Np, length(find(antiVs[4,:] .> antiVs[1,:]))/Na)

@printf("\n\n\n---\n\nmypars contains all the parameters for the new run with the scaled up dt, tau, etc.\n\n")


In [ ]:
mypars[:dt]

In [ ]:
# Print out mypars in full
@printf("\n\nmypars = Dict(\n")
for (k,v) in mypars
    @printf(":%s => ", k); print(v); print(",\n")
end
@printf(")\n\n")

# Make a set of reduced nsteps params and run with them

In [ ]:

mypars = Dict(
:init_add               =>          0,
:const_add              =>          0,
:noise                  =>          Any[],
:input                  =>          0,
:seedrand               =>          1510340076445,
:start_anti             =>          [-0.5, -0.5, -0.5, -0.5],
:start_pro              =>          [-0.5, -0.5, -0.5, -0.5],
:rule_and_delay_period  =>          1.2,
:rule_and_delay_periods =>          [1.2],
:target_period          =>          0.3,
:target_periods         =>          [0.3],
:post_target_period     =>          0.3,
:post_target_periods    =>          [0.3],
:anti_rule_strength     =>          0.054,
:U_rest                 =>          0,
:theta                  =>          0.05,
:beta                   =>          0.5,
:g_leak                 =>          1,
:nsteps                 =>          301,
:dt                     =>          0.024,
:tau                    =>          0.09,
:right_light_excitation =>          0.49924152955481954,
:opto_strength          =>          0.85,
:opto_periods           =>          String[
                                    "trial_start" "trial_start"; 
                                    "trial_start" "trial_end"; 
                                    "trial_start" "target_start/2"; 
                                    "target_start/2" "target_start"; 
                                    "target_start" "target_end"],
:opto_targets          =>           [
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7],
:theta2 => 0.15,
:theta1 => 0.05,
:sigma => 0.01,
:cbeta => 0.04,
:sW => 0.6416875048295452,
:hW => 0.054701836208134846,
:dW => 0.1267124266934907,
:vW => -1.588850577499782,
:constant_excitation => -0.37242520737694207,
:const_pro_bias => 0.04366897857834884,
:target_period_excitation => 0.15315254453690974,
:right_light_pro_extra => 0,
:pro_rule_strength => 0.05,
:nPro => 100,
:nAnti => 100,
)


In [ ]:

extra_pars = Dict(
:plot_list        =>   [1:5;], 
:plot_conditions  =>   true,
:verbose          =>   true,
:opto_periods     =>   String["trial_start" "trial_start"],
:opto_targets     =>   [0.9,  0.7],
:opto_times       =>   ["trial_start", "trial_start"],       # This one is for run_ntrials
:seedrand         =>   Int64(round(time()*1000)),
)


# proVs, antiVs, pro_fullV, anti_fullV, opto_fraction, pro_input, anti_input = run_ntrials(1000, 1000; 
#    merge(mypars, extra_pars)...);


JJ(mypars[:nPro], mypars[:nAnti]; merge(mypars, extra_pars)...)[1]


In [ ]:
extra_pars[:search_range] = 0.4
extra_pars

## Try a search

`Temp/answers.jld`   contains the result of taking the result of scaling_up_dt.jl, and then optimizing it on control alone, with Pro target=0.9, Anti target = 0.7.  It optimizes very very well.

In [2]:
#@include_me reduced_farm.jl

include("pro_anti.jl")



load_run

In [89]:
@load("FarmFields/farm_C3_0027.jld")

18-element Array{Symbol,1}:
 :args      
 :bbox      
 :cost      
 :cpm_traj  
 :dA        
 :dP        
 :extra_pars
 :ftraj     
 :hA        
 :hBA       
 :hBP       
 :hP        
 :mypars    
 :nAnti     
 :nPro      
 :pars      
 :seed      
 :traj      

In [90]:
[args pars]

9×2 Array{Any,2}:
 "dW"                         0.198366 
 "target_period_excitation"   0.212294 
 "const_pro_bias"             0.0247642
 "hW"                        -0.277058 
 "sW"                         1.54588  
 "vW"                        -1.41822  
 "sigma"                      0.1      
 "constant_excitation"       -0.633155 
 "right_light_excitation"     0.452452 

In [141]:
mypars[:tau]

0.09

QPixmap::scaled: Pixmap is a null pixmap


In [ ]:
#@include_me reduced_farm.jl


mypars = Dict(
:init_add               =>          0,
:const_add              =>          0,
:noise                  =>          Any[],
:input                  =>          0,
:seedrand               =>          1510340076445,
:start_anti             =>          [-0.5, -0.5, -0.5, -0.5],
:start_pro              =>          [-0.5, -0.5, -0.5, -0.5],
:rule_and_delay_period  =>          1.2,
:rule_and_delay_periods =>          [1.2],
:target_period          =>          0.3,
:target_periods         =>          [0.3],
:post_target_period     =>          0.3,
:post_target_periods    =>          [0.3],
:anti_rule_strength     =>          0.054,
:U_rest                 =>          0,
:theta                  =>          0.05,
:beta                   =>          0.5,
:g_leak                 =>          1,
:nsteps                 =>          301,
:dt                     =>          0.024,
:tau                    =>          0.09,
:right_light_excitation =>          0.49924152955481954,
:opto_strength          =>          0.85,
:opto_periods           =>          String[
                                    "trial_start" "trial_start"; 
                                    "trial_start" "trial_end"; 
                                    "trial_start" "target_start/2"; 
                                    "target_start/2" "target_start"; 
                                    "target_start" "target_end"],
:opto_targets          =>           [
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7; 
                                    0.9 0.5; 
                                    0.9 0.7],
:theta2 => 0.15,
:theta1 => 0.05,
:sigma => 0.01,
:cbeta => 0.04,
:sW => 0.6416875048295452,
:hW => 0.054701836208134846,
:dW => 0.1267124266934907,
:vW => -1.588850577499782,
:constant_excitation => -0.37242520737694207,
:const_pro_bias => 0.04366897857834884,
:target_period_excitation => 0.15315254453690974,
:right_light_pro_extra => 0,
:pro_rule_strength => 0.05,
:nPro => 100,
:nAnti => 100,
)


In [ ]:
#@include_me  reduced_farm.jl


extra_pars = Dict(
:plot_list        =>   [], 
:plot_conditions  =>   true,
:verbose          =>   true,
:opto_periods     =>   String["trial_start" "trial_start-0.1"],
:opto_targets     =>   [0.9,  0.7],
:opto_times       =>   ["trial_start", "trial_start-0.1"],       # This one is for run_ntrials
:seedrand         =>   Int64(round(time()*1000)),  # 1510426840370  Works wonders with search_range = 0.01
:cbeta            =>   0.001,
:search_range     =>   1.2,
)


search_conditions = Dict(   # :param    default_start   search_box  bound_box
:vW     =>                   [mypars[:vW],                       [-0.5, 0.5],  [-3,   3]], 
:hW     =>                   [mypars[:hW],                       [-0.5, 0.5],  [-3,   3]],
:dW     =>                   [mypars[:dW],                       [-0.5, 0.5],  [-3,   3]],
:sW     =>                   [mypars[:sW],                       [0,    0.5],  [0,    3]],
:sigma  =>                   [0.11,                              [0.1,  0.2],  [0.1, 0.4]],
:constant_excitation      => [mypars[:constant_excitation],      [-1,     1],  [-2,   2]], 
:target_period_excitation => [mypars[:target_period_excitation], [0.001,0.5],  [0     4]],
:right_light_excitation   => [mypars[:right_light_excitation],   [0.05, 0.5],  [0.05, 4]],
:const_pro_bias           => [mypars[:const_pro_bias],           [-0.5, 0.5],  [-2,   2]],
# :opto_strength            => [mypars[:opto_strength],            [0.7, 0.99],  [0,    1]],
)




In [ ]:
#@include_me  reduced_farm.jl


search_range = extra_pars[:search_range]; 

fbasename = "FarmFields/farm_C3_"

while true
    args = []; seed = []; bbox = Dict()
    for k in keys(search_conditions)
        search_box = search_conditions[k][2]
        args = [args; String(k)]
        myseed = search_conditions[k][1] + search_range*(rand()-0.5)*diff(search_box); myseed = myseed[1]
        if myseed > search_box[2]; myseed = search_box[2]; end
        if myseed < search_box[1]; myseed = search_box[1]; end
        seed = [seed ;  myseed]
        # seed = [seed ; search_conditions[k][1]]
        bbox = merge(bbox, Dict(k => Array{Float64}(search_conditions[k][3])))
    end
    args = Array{String}(args)
    seed = Array{Float64}(seed)



    func =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=false, 
    merge(merge(mypars, extra_pars), Dict(params))...)[1]

    try
        pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(seed, args, bbox, func, 
            start_eta = 0.1, tol=1e-12, 
            verbose=true, verbose_every=1, maxiter=1000)


        cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(10000, 10000; verbose=false, 
        make_dict(args, pars, merge(merge(mypars, extra_pars)))...)

        myfilename = next_file(fbasename, 4)
        myfilename = myfilename*".jld"
        # write file
        save(myfilename, Dict("nPro"=>mypars[:nPro], "nAnti"=>mypars[:nAnti], 
        "mypars"=>mypars, "extra_pars"=>extra_pars, "args"=>args, "seed"=>seed, "bbox"=>bbox, 
        "pars"=>pars, "traj"=>traj, "cost"=>cost, "cpm_traj"=>cpm_traj, "ftraj"=>ftraj,
        "hP"=>hP, "hA"=>hA, "dP"=>dP, "dA"=>dA, "hBP"=>hBP, "hBA"=>hBA))
    catch
        @printf("\n\nWhoopsety, unkown error!   Trying new random seed.\n\n")
    end
end

In [ ]:
effective_search_box = Dict()
for k in keys(search_conditions)
    search_box = search_conditions[k][2]
    ebox = zeros(1,2)
    ebox[1] = search_conditions[k][1] - 0.5*diff(search_box)[1]
    ebox[2] = search_conditions[k][1] + 0.5*diff(search_box)[1]
    effective_search_box[k] = ebox
end

effective_search_box

# Analyzing runs from reduced_farm.jl

In [ ]:
#@include_me  analyze_reduced_farm.jl

include("pro_anti.jl")

#
#  We're assuming that results of reduced_farm.jl were already generated.
#

In [112]:
#@include_me  analyze_reduced_farm.jl

"""
function show_run(run_name, ntrials=100; further_pars...)

Given a string representing the filename of a run generated in reduced_farm.jl
(for example, "farm_C3_0027.jld"), tries to load that file from within "FarmFields/"
runs it to plot 30 Pro trials in figure 1 and 30 Anti trials in figure 2,
and indicates in those figures the corresponding number of binarized hits.

Any further parameters override the contents of the farm and are passed on to 
run_ntrials(), so for example you can set rule_and_delay_period=0.2
"""
function show_run(run_name, ntrials=100; further_pars...)

    pygui(true)
    figure(1); clf();
    figure(2); clf();

    d = load("FarmFields/" * run_name)

    extra_pars = d["extra_pars"]
    mypars     = d["mypars"]
    args       = d["args"]
    pars       = d["pars"]

    delete!(extra_pars, :plot_list)   # we want our own plot_list
    delete!(extra_pars, :seedrand)    # don't use the farm's seedrand. (Could change this if wanted)

    # Add or replace alternative period values if needed
    # if rule_and_delay_period != nothing; extra_pars[:rule_and_delay_period] = rule_and_delay_period; end;
    # if target_period         != nothing; extra_pars[:target_period]         = target_period;         end;
    # if post_target_period    != nothing; extra_pars[:post_target_period]    = post_target_period;    end;

    # Run it
    proVs, antiVs = run_ntrials(ntrials, ntrials; plot_list=[1:30;], 
        merge(make_dict(args, pars, merge(mypars, extra_pars)), Dict(further_pars))...)[1:2]

    # Add titles with the binarized hit rates
    hBP = 100*length(find(proVs[1,:]  .> proVs[4,:])) /size(proVs, 2)
    hBA = 100*length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2)

    figure(1); subplot(3,1,1); title(@sprintf("%s: binarized Pro  hits = %.1f %%\n", run_name, hBP))
    figure(2); subplot(3,1,1); title(@sprintf("%s: binarized Anti hits = %.1f %%\n", run_name, hBA))
    
    return hBP, hBA
end


show_run

In [144]:
#@include_me analyze_reduced_farm.jl

# IDENTIFY GOOD RUNS FROM C3 FARM

I = results = files = 0

recompute_me = false; if recompute_me
    results = zeros(0,4)
    files = []
    for f in filter(x -> startswith(x, "farm_C3_"), readdir("FarmFields/"))
        hBP, hBA, dP, dA= load("FarmFields/" * f, "hBP", "hBA", "dP", "dA")
        results = [results; [hBP hBA dP dA]]; 
        files   = [files ; f]
    end
    
    I = find((results[:,1] .> 0.9) .& (results[:,2] .> 0.65) .& (results[:,3].>0.85) .& (results[:,4].>0.85))

    @save "Temp/C3_results.jld" results files I
else
    I, results, files = load("Temp/C3_results.jld", "I", "results", "files");
end


([10, 21, 23, 26, 27, 29, 40, 41, 52, 60  …  1533, 1538, 1548, 1551, 1552, 1557, 1558, 1567, 1576, 1582], [0.5992 0.3987 0.48625 0.477268; 0.492 0.5039 0.998939 0.999669; … ; 0.6679 0.3381 0.67854 0.709378; 0.9801 0.0021 0.355964 0.0506895], Any["farm_C3_0001.jld", "farm_C3_0002.jld", "farm_C3_0003.jld", "farm_C3_0004.jld", "farm_C3_0005.jld", "farm_C3_0006.jld", "farm_C3_0007.jld", "farm_C3_0008.jld", "farm_C3_0009.jld", "farm_C3_0010.jld"  …  "farm_C3_1579.jld", "farm_C3_1580.jld", "farm_C3_1581.jld", "farm_C3_1582.jld", "farm_C3_1583.jld", "farm_C3_1584.jld", "farm_C3_1585.jld", "farm_C3_1586.jld", "farm_C3_1587.jld", "farm_C3_1588.jld"])

In [146]:
#@include_me  analyze_reduced_farm.jl

"""
    go_over_good_runs(files, I)

Given a list of files, and a list of indices into those files, iterates over the indicated
filenames, running show_run() on them, and then asking for user input.
"""
function go_over_good_runs(files, I)
    for k=1:length(I)
        show_run(files[I[k]])

        @printf("Any key to continue, q to quit: ")
        ans = chomp(readline())
        if ans=="q" || ans=="Q"
            break
        end
    end
end


go_over_good_runs

In [147]:
go_over_good_runs(files, I)

Any key to continue, q to quit: STDIN> 
Any key to continue, q to quit: STDIN> 
Any key to continue, q to quit: STDIN> q


In [140]:
#@include_me analyze_reduced_farm.jl

# show_run("farm_C3_0027.jld", 400; opto_times=["trial_start", "trial_end"], opto_strength=0.75)
# show_run("farm_C3_0027.jld", 400; opto_times=["target_start-0.5", "target_start"], opto_strength=0.4, 
#    const_pro_bias=0.02476, anti_rule_strength=0.05)
show_run("farm_C3_0027.jld", 400; opto_times=["target_start+0.15", "target_end"], opto_strength=0.4, 
    const_pro_bias=0.02476, anti_rule_strength=0.054)

(93.5, 63.5)

In [152]:
#@include_me analyze_reduced_farm.jl

#  TARGET PERIOD OPTO

run_name = "farm_C3_0027.jld"
recompute_me = false; if recompute_me
    ostrengths = 0.5:0.1:1
    phits      = zeros(1, length(ostrengths))
    ahits      = zeros(1, length(ostrengths))

    for i=1:length(ostrengths)
        phits[i], ahits[i] = show_run(run_name, 4000; 
        opto_times=["target_start+0.016", "target_end+0.016"], opto_strength=ostrengths[i])
        @printf("ostrength=%g, phits=%.1f, ahits=%.1f\n", ostrengths[i], phits[i], ahits[i])
    end

    @save("Temp/opto_target.jld", ostrengths, ahits, phits)
else
    ostrengths, ahits, phits = load("Temp/opto_target.jld", "ostrengths", "ahits", "phits")
end

figure(3);clf();
plot(ostrengths, phits[:], ".-", ostrengths, ahits[:], ".-")
legend(["pro", "anti"])
xlabel("opto strength")
ylabel("binarized hits")
title(run_name * " : TARGET PERIOD opto")
grid("on")


#  DELAY PERIOD OPTO  ---------

recompute_me = false; if recompute_me
    ostrengths = 0.5:0.1:1
    phits      = zeros(1, length(ostrengths))
    ahits      = zeros(1, length(ostrengths))

    for i=1:length(ostrengths)
        phits[i], ahits[i] = show_run(run_name, 4000; 
        opto_times=["target_start-0.5", "target_start"], opto_strength=ostrengths[i])
        @printf("ostrength=%g, phits=%.1f, ahits=%.1f\n", ostrengths[i], phits[i], ahits[i])
    end
    @save("Temp/opto_delay.jld", ostrengths, ahits, phits)
else
    ostrengths, ahits, phits = load("Temp/opto_delay.jld", "ostrengths", "ahits", "phits")
end

figure(4);clf();
plot(ostrengths, phits[:], ".-", ostrengths, ahits[:], ".-")
legend(["pro", "anti"])
xlabel("opto strength")
ylabel("binarized hits")
title(run_name * " : last 500 ms of DELAY PERIOD opto")
grid("on")

In [ ]:
search_conditions

In [ ]:
@load "Temp/farm_C1_0001.jld"

mypars[:seedrand] = Int64(round(time()*1000))

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(10000, 10000; verbose=false, 
make_dict(args, pars, merge(merge(mypars, extra_pars)))...)

[hBP  hBA]

In [ ]:
[hP hA dP dA hBP hBA]

In [6]:
q = chomp(readline())
q

STDIN> q


"q"

In [ ]:
show_run("farm_C3_0002.jld")


In [ ]:
figure(1); subplot(3,1,1); title(@sprintf("%s: binarized Pro  hits = %.1f %%\n", run_name, 97.3))


In [ ]:
# @save "Temp/answers.jld" 
# @load "Temp/answers.jld"

In [ ]:
@load "Temp/answers3.jld"

In [ ]:
[args pars seed]

In [ ]:
pygui(true)
figure(1); clf();

delete!(extra_pars, :plot_list)
delete!(extra_pars, :seedrand)

more_pars = Dict(
# :post_target_period => 0.25,
:rule_and_delay_period => 2,
)
proVs, antiVs = run_ntrials(1000, 1000; plot_list=[1:10;], make_dict(args, pars, merge(merge(mypars, extra_pars), more_pars))...)[1:2]

@printf("Binarized Pro hits = %.1f %%\n", 100*length(find(proVs[1,:] .> proVs[4,:]))/size(proVs,2))
@printf("Binarized Anti hits = %.1f %%\n", 100*length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2))

In [ ]:
search_conditions

In [ ]:
include("pro_anti.jl")

In [ ]:
bbox[:dW]

In [ ]:
[1, [2,3], 4]


In [ ]:
#  size(proValls)  (4, 301, 100, 5)

figure(3); clf(); N=40

t = 0:size(proValls, 2)-1; t = model_params[:dt]*t

figure(3); clf();
for i=1:N 
    plot_PA(t, 0, proValls[:,:,i,1], fignum=3, plot_Us=false,
    rule_and_delay_period=model_params[:rule_and_delay_period], target_period=model_params[:target_period],
    post_target_period = model_params[:post_target_period])
end

figure(4); clf();
for i=1:N 
    plot_PA(t, 0, antiValls[:,:,i,1], fignum=4, plot_Us=false,
    rule_and_delay_period=model_params[:rule_and_delay_period], target_period=model_params[:target_period],
    post_target_period = model_params[:post_target_period])
end


In [ ]:
print(model_params)

In [ ]:
# Define core model parameters
model_params = Dict(
:dt     =>  0.002, 
:tau    =>  0.02, 
:vW     =>  -1.58,
:hW     =>  -0.05,
:sW     =>  0,
:dW     =>  0,
:nsteps =>  301, 
:noise  =>  [], 
:sigma  =>  0.08, 
:input  =>  0, 
:g_leak =>  1, 
:U_rest =>  0,
:theta  =>  0.05, 
:beta   =>  0.5, 
:constant_excitation      => 0, 
:anti_rule_strength       => 0.05,
:pro_rule_strength        => 0.05, 
:target_period_excitation => 0,
:right_light_excitation   => 0.6, 
:right_light_pro_extra    => 0,
:const_add                => 0, 
:init_add                 => 0, 
:rule_and_delay_period    => 0.2,
:target_period            => 0.1,
:post_target_period       => 0.1,
:const_pro_bias           => 0.0427,
:nPro                     => 100,
:nAnti                    => 100,
:theta1                   => 0.05,
:theta2                   => 0.15,
:opto_strength  => .9,
:opto_periods   => [
    0               0 ; 
    0              "trial_end" ;
    "target_start/2"  "target_start";
    "target_start"    "target_end"],  
# set of opto conditions, in seconds, with 0 the start 
# of the trial (i.e. start of rule_and_delay_period), anything before 0 or after end of trial gets ignored.
#:opto_targets   => [.75 .73;.77 .58;.75 .74;.72 .66;.73 .75] 
:opto_targets => [
    .9      .7; 
    .9      .5; 
    .9      .55; 
    .9      .7;
    ],  # first column is frachit Pro, next column is Anti, rows are conditions
# The "conditions" correspond to the rows of opto_periods.
);


# ======= ARGUMENTS AND SEED VALUES:
args = ["sW", "vW", "hW", "dW", "constant_excitation", "right_light_excitation", "target_period_excitation"]
seed = [0.001,  -1.58,   -0.05,  0.001,    0.001,                0.6,                       0.001]   
args = [args ; ["const_pro_bias", "sigma","opto_strength", "pro_rule_strength", "anti_rule_strength"]];
seed = [seed ; [  0.0427,            .05,      .9,              0.05,                0.05]];

# ======= BOUNDING BOX:
bbox = Dict(:sW=>[0 3], :vW=>[-3 3], :hW=>[-3 3], :dW=>[-3 3], :constant_excitation=>[-2 2],
:right_light_excitation=>[0.05 4], :target_period_excitation=>[0 4], :const_pro_bias=>[-2 2],
:sigma=>[0.01 0.2],:opto_strength=>[0 1]);

# ======== SEARCH ZONE:
sbox = Dict(:sW=>[0 .5], :vW=>[-.5 .5], :hW=>[-.5 .5], :dW=>[-.5 .5],
:constant_excitation=>[-.5 .5], :right_light_excitation=>[0.05 .5], :target_period_excitation=>[0.001 .5],:const_pro_bias=>[-.5 .5], :sigma=>[0.02 0.19],:opto_strength=>[.7 .99]);

# define a few hyper parameters
cbetas = [0.04];
rule_and_delay_periods = [0.2];
post_target_periods    = [0.1];
num_eval_runs           = 100;
num_optimize_iter       = 2000;
num_optimize_restarts   = 1;


cb = 0.04

# figure out initial seed for random number generator
sr = convert(Int64, round(time()))
srand(sr);

mypars = merge(model_params, Dict(:opto_times => ["target_start", "target_end"], :rule_and_delay_period=>0.2,
:anti_rule_strength=>0.06))
# mypars = merge(model_params, Dict(:opto_times => [0 0], :rule_and_delay_period=>0.1,
# :anti_rule_strength=>0.06))
pygui(true)
proVs, antiVs, pro_fullV, anti_fullV, opto_fraction, pro_input, anti_input = run_ntrials(15, 15; 
plot_list=[1:15;], profig=1, antifig=2, opto_units = 1:4, mypars...)

# define opto function with just value output
func =  (;params...) -> JJ(model_params[:nPro], model_params[:nAnti]; 
    rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], 
    post_target_periods=post_target_periods, seedrand=sr, cbeta=cb, verbose=true, 
    merge(mypars, Dict(params))...)[1]


# cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(model_params[:nPro], model_params[:nAnti]; 
#     rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], 
#     post_target_periods=post_target_periods, seedrand=sr, cbeta=cb, verbose=true, model_params...)
    

In [ ]:
@printf("Going with seed = "); print_vector_g(seed); print("\n")
pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(seed, args, bbox, func, 
    start_eta = 1, tol=1e-12, verbose=true, verbose_every=1, maxiter=num_optimize_iter)
@printf("Came out with cost %g and pars = ", cost); print_vector_g(pars); print("\n\n")


In [ ]:
# run optimization with all parameters
@printf("Going with seed = "); print_vector_g(seed); print("\n")
pars, traj, cost, cpm_traj, ftraj = bbox_Hessian_keyword_minimization(seed, args, bbox, func, 
    start_eta = 1, tol=1e-12, verbose=true, verbose_every=1, maxiter=num_optimize_iter)
@printf("Came out with cost %g and pars = ", cost); print_vector_g(pars); print("\n\n")

    # get gradient and hessian at end of optimization 
    value, grad, hess = keyword_vgh(func, args, pars)

    # define function with all outputs, evaluate on training noise
    t_standard_func =  (;params...) -> JJ_opto(model_params[:nPro], model_params[:nAnti]; rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], post_target_periods=post_target_periods, seedrand=sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    # run opto model with all outputs, evaluate on training noise
    t_opto_scost, t_opto_scost1, t_opto_scost2, t_opto_hitsP,t_opto_hitsA, t_opto_diffsP, t_opto_diffsA, t_opto_bP, t_opto_bA = t_standard_func(;make_dict(args, pars, model_params)...)
    
   ## evaluate for long form info
    # reset random number generator for testing purposes
    test_sr = convert(Int64, round(time()))
    srand(test_sr); 

    # define function with all outputs, evaluate on test noise
    standard_func =  (;params...) -> JJ_opto(num_eval_runs, num_eval_runs; rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], post_target_periods=post_target_periods, seedrand=test_sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    # run opto model with all outputs, evaluate on test noise
    opto_scost, opto_scost1, opto_scost2, opto_hitsP,opto_hitsA, opto_diffsP, opto_diffsA, opto_bP, opto_bA = standard_func(;make_dict(args, pars, model_params)...)

    # define non-opto model with all outputs, to check opto model, evaluate on test noise
    standard_func =  (;params...) -> JJ(num_eval_runs, num_eval_runs; rule_and_delay_periods=rule_and_delay_periods, theta1=model_params[:theta1], theta2=model_params[:theta2], post_target_periods=post_target_periods, seedrand=test_sr, cbeta=cb, verbose=false, merge(model_params, Dict(params))...)

    # run non-opto model with all outputs, evaluate on test noise
    scost, scost1, scost2, hitsP,hitsA, diffsP, diffsA = standard_func(;make_dict(args, pars, model_params)...)
 
   ## Save this run out to a file
    # get filename
    myfilename = next_file(fbasename, 4)
    myfilename = myfilename*".mat"
    # write file
    matwrite(myfilename, Dict("args"=>args, "myseed"=>myseed, "dista"=>dista, "pars"=>pars, "traj"=>traj, "cost"=>cost, "cpm_traj"=>cpm_traj, "nPro"=>model_params[:nPro], "nAnti"=>model_params[:nAnti], "sr"=>sr, "cb"=>cb, "theta1"=>model_params[:theta1], "theta2"=>model_params[:theta2],"value"=>value,"grad"=>grad, "hess"=>hess, "scost"=>scost, "scost1"=>scost1, "scost2"=>scost2, "hitsP"=>hitsP,"hitsA"=>hitsA, "diffsP"=>diffsP, "diffsA"=>diffsA, "model_params"=>ascii_key_ize(model_params), "bbox"=>ascii_key_ize(bbox), "sbox"=>ascii_key_ize(sbox), "rule_and_delay_periods"=>rule_and_delay_periods, "post_target_periods"=>post_target_periods, "opto_scost"=>opto_scost, "opto_scost1"=>opto_scost1, "opto_scost2"=>opto_scost2, "opto_hitsP"=>opto_hitsP, "opto_hitsA"=>opto_hitsA, "opto_diffsP"=>opto_diffsP, "opto_diffsA"=>opto_diffsA,"test_sr"=>test_sr,"opto_bP"=>opto_bP, "opto_bA"=>opto_bA, "t_opto_scost"=>t_opto_scost, "t_opto_scost1"=>t_opto_scost1, "t_opto_scost2"=>t_opto_scost2, "t_opto_hitsP"=>t_opto_hitsP, "t_opto_hitsA"=>t_opto_hitsA, "t_opto_diffsP"=>t_opto_diffsP, "t_opto_diffsA"=>t_opto_diffsA,"t_opto_bP"=>t_opto_bP, "t_opto_bA"=>t_opto_bA  ))
end
end